In [1]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my
import shared.control_center as CC

In [2]:
cis = fct.c_vir(0, my.Mvir_NFW)
cis1 = fct.c_vir(4, my.Mvir_NFW)
print(cis, cis1)

14.928729169804287
14.928729169804287
16.758793969849247 0.04258828167400367


# Various tests on individual functions

## Number integral

In [ ]:
FDarr = np.array([0.3,0.4,0.1,0.01])
p_arr = np.array([0.2, 0.1, 5., 8.]) * my.T_nu_eV.value

order = p_arr.argsort()
p_sort, FD_sort = p_arr[order], FDarr[order]
# print(p_sort, FD_sort)

n_int = np.trapz(p_sort**2 * FD_sort, p_sort)
print(n_int)

n_sum = np.sum(p_sort**2 * FD_sort)
print(n_sum)

## Integral for cosmic time

In [ ]:
def t_integrand_a(a):

    # original H0 in units ~[1/s], we only need the value
    H0 = my.H0.to(unit.s**-1).value

    a_dot = np.sqrt(my.Omega_m0/a**3 + my.Omega_L0)*H0*a
    t_int = 1/a_dot

    return t_int

t, err = quad(t_integrand_a, 0, 1)
t_uni, err_uni = (t*unit.s).to(unit.Gyr), (err*unit.s).to(unit.Gyr)
print(t_uni, err_uni)

In [ ]:
def t_integrand_z(z):

    # original H0 in units ~[1/s], we only need the value
    H0 = my.H0.to(unit.s**-1).value

    a_dot = np.sqrt(my.Omega_m0*(1+z)**3 + my.Omega_L0)*H0*(1+z)
    t_int = 1/a_dot

    return t_int

t, err = quad(t_integrand_z, 0, np.inf)
t_uni, err_uni = (t*unit.s).to(unit.Gyr), (err*unit.s).to(unit.Gyr)
print(t_uni, err_uni)

## Initial velocity limits

In [ ]:
m_sim_eV = 0.05*unit.eV
low_kpc, upp_kpc = fct.velocity_limits_of_m_nu(0.01, 10., m_sim_eV, mode='kpc/s')
print(low_kpc,upp_kpc)

m_sim_eV = 0.05*unit.eV
low_km, upp_km = fct.velocity_limits_of_m_nu(0.01, 10., m_sim_eV, mode='km/s')
print(low_km,upp_km)

## Simulated velocities in [kpc/s] to momenta in [eV] for any (neutrino) mass

In [ ]:
u_sim = np.load(f'neutrino_vectors/nu_4586.npy')[-1,3:6]
m_sim_eV = 0.05*unit.eV
m_target_eV = 0.05*unit.eV

p, y = fct.u_to_p_eV(u_sim, m_sim_eV, m_target_eV)
print(p, y)

## 1/hc to cm^-1/eV

In [ ]:
hc_neg1 = (1/const.h/const.c).to(1/unit.cm/unit.eV)
print(hc_neg1)

## Table 1

In [6]:
z_test = 4
print('R_vir:', fct.R_vir(z_test, my.Mvir_NFW))
print('scale_radius:', fct.scale_radius(z_test, my.Mvir_NFW))

R_vir: 81.78036756845792 kpc
14.928729169804287
scale_radius: 1920.255158319231 kpc


## Critical Density

In [ ]:
crit = fct.rho_crit(0)
print('Check if this matches critical density of universe today:')
print(crit.to(unit.kg/unit.m**3))

## Unit Conversion Tests

In [ ]:
print(my.T_nu.to(unit.eV, unit.temperature_energy()))

## Derivative vector values

In [ ]:
z = 0
x_i = np.array([8.5,0.,0.])*unit.kpc

t2 = fct.dPsi_dxi_NFW(x_i, z, my.rho0_NFW, my.Mvir_NFW)
print(type(t2), t2)

## Time Variable s

In [ ]:
s = fct.s_of_z(4)
print('Value of time variable s in seconds at redhshift 4','\n', s)

h_s = my.H0.to(1/unit.s)
print('Age of universe in seconds:','\n', 1/h_s)

In [ ]:
s = fct.s_of_z(4)
print('Value of time variable s in seconds at redhshift 4','\n', s)

h_s = my.H0.to(1/unit.s)
print('Age of universe in seconds:','\n', 1/h_s)

## Plot for s and z relation

In [ ]:
zeds = np.geomspace(1e-10, CC.Z_STOP, CC.Z_AMOUNT)

ss = np.array([fct.s_of_z(z) for z in zeds]) * my.H0.to(unit.s**-1).value


plt.semilogx(1+zeds, ss)
plt.title('Shape of integral for s(z)')
plt.xlabel('redshift')
plt.ylabel('time variable s [1/H0]')
plt.savefig('check_plots/s_of_z_integral.pdf')

## Initial Velocities

In [ ]:
from backtracing import draw_ui

# Draw initial velocities.
ui = draw_ui(CC.PHIs, CC.THETAs, CC.Vs)*my.Uunit
ui = ui.to(unit.km/unit.s).value

ux = ui[:,0]
uy = ui[:,1]
uz = ui[:,2]

x = np.arange(len(ux))
y = np.arange(len(uy))
z = np.arange(len(uz))

plt.plot(x, ux, label='x-axis', )
plt.plot(y, uy, label='y-axis', )
plt.plot(z, uz, label='z-axis', alpha=0.5)
plt.title(f'Initial Velocities - {CC.NR_OF_NEUTRINOS} neutrinos')
plt.xlabel('Neutrino number')
plt.ylabel('Velocity w.r.t. axis in [km/s]')
plt.legend()
plt.savefig('check_plots/initial_velocities.pdf')

## Simpy derivative

In [ ]:
### This is for the whole expression as in eqn. (A.5) and using sympy

# m = np.minimum(r0, r_vir)
# M = np.maximum(r0, r_vir)

# r = sympy.Symbol('r')

# prefactor = -4*np.pi*unit.G*rho_0*r_s**2
# term1 = np.log(1 + m/r_s) / (r/r_s)
# term2 = r_vir/M / (1 + r_vir/r_s)
# Psi = prefactor * (term1 - term2)

## derivative w.r.t any axis x_i with chain rule
# dPsi_dxi = sympy.diff(Psi, r) * x_i / r0

## fill in r values
# fill_in_r = sympy.lambdify(r, dPsi_dxi, 'numpy')
# derivative_vector = fill_in_r(r0)